# Mise en forme des données dans un dataframe

## Création du dataframe

In [12]:
import pandas as pd

In [13]:
dfQuestions = pd.DataFrame(
        {
            "id"  : [],
            "Groupe Parlementaire" : [],
            "Nom du député" : [],
            "Question" : []
        })

In [14]:
cols =  ["id",
            "Groupe Parlementaire",
            "Nom du député",
            "Question"]

## Récupération des données d'une question

#### Imports et fonctions utilitaires

In [17]:
!pip install webdriver-manager
!pip install -q lxml

import bs4
import lxml
import pandas
import urllib
import re

from urllib import request

In [18]:
def pageurl(x) : 
    return "https://questions.assemblee-nationale.fr/q16/16-"+str(x)+"QE.htm"

#### Récupération d'une question

In [19]:
k = 1

In [20]:
source_code = request.urlopen(pageurl(k)).read()
page = bs4.BeautifulSoup(source_code)
question0 = page.find("div", {"class" : "question"})
question = question0.find("p")
question = question.text.strip()

#### Récuperation des données du parlementaire

In [21]:
lien0 = page.find("span", {"class" : "question_big_content"})
lien = lien0.find("a").get("href")

In [22]:
source_code_parlementaire = request.urlopen(lien).read()
page_parlementaire = bs4.BeautifulSoup(source_code_parlementaire)


In [23]:
groupe0 = page_parlementaire.find("a", {"class" : "h4 _colored link"})
groupe = groupe0.text.strip()

nom0 = page_parlementaire.find("h1", {"class" : "h1 _mt-small"})
nom = nom0.text.strip()

#### Ajout de la donnée au dataframe

In [24]:
dfQuestion = pd.DataFrame([[k, groupe, nom, question]], columns = cols)

In [25]:
dfQuestions = pd.concat([dfQuestions, dfQuestion], ignore_index = True)
dfQuestions

,id,Groupe Parlementaire,Nom du député,Question
0,1.0,"Libertés, Indépendants, Outre-mer et Territoires",M. Christophe Naegelen,M. Christophe Naegelen interroge M. le ministr...


## Création du Data Frame final

In [26]:
import pandas as pd
import bs4
import lxml
import pandas
import urllib
import re

from urllib import request

def pageurl(x) : 
    return "https://questions.assemblee-nationale.fr/q16/16-"+str(x)+"QE.htm"

In [27]:
n = 50

In [28]:
dfQuestions = pd.DataFrame(
        {
            "id"  : [],
            "Groupe Parlementaire" : [],
            "Nom du député" : [],
            "Question" : []
        })

for k in range(1, n+1) : 
    source_code = request.urlopen(pageurl(k)).read()
    page = bs4.BeautifulSoup(source_code)
    question0 = page.find("div", {"class" : "question"})
    question = question0.find("p")
    question = question.text.strip()
    
    lien0 = page.find("span", {"class" : "question_big_content"})
    lien = lien0.find("a").get("href")

    source_code_parlementaire = request.urlopen(lien).read()
    page_parlementaire = bs4.BeautifulSoup(source_code_parlementaire)

    groupe0 = page_parlementaire.find("a", {"class" : "h4 _colored link"})
    groupe = groupe0.text.strip()

    nom0 = page_parlementaire.find("h1", {"class" : "h1 _mt-small"})
    nom = nom0.text.strip()

    dfQuestion = pd.DataFrame([[k, groupe, nom, question]], columns = cols)
    dfQuestions = pd.concat([dfQuestions, dfQuestion], ignore_index = True)

dfQuestions

,id,Groupe Parlementaire,Nom du député,Question
0,1.0,"Libertés, Indépendants, Outre-mer et Territoires",M. Christophe Naegelen,M. Christophe Naegelen interroge M. le ministr...
1,2.0,Les Républicains,M. Pierre Cordier,M. Pierre Cordier appelle l'attention de Mme l...
2,3.0,Démocrate (MoDem et Indépendants),M. Frédéric Petit,M. Frédéric Petit appelle l'attention de M. le...
3,4.0,Renaissance,Mme Marie-Pierre Rixain,Mme Marie-Pierre Rixain appelle l'attention de...
4,5.0,"Libertés, Indépendants, Outre-mer et Territoires",M. Christophe Naegelen,M. Christophe Naegelen interroge M. le ministr...
5,6.0,Gauche démocrate et républicaine - NUPES,M. Stéphane Peu,M. Stéphane Peu interroge M. le ministre de l'...
6,7.0,La France insoumise - Nouvelle Union Populaire...,M. Paul Vannier,M. Paul Vannier alerte M. le ministre de l'édu...
7,8.0,Les Républicains,Mme Alexandra Martin (Alpes-Maritimes),Mme Alexandra Martin (Alpes-Maritimes) appelle...
8,9.0,Renaissance,Mme Marie-Pierre Rixain,Mme Marie-Pierre Rixain attire l'attention de ...
9,10.0,Renaissance,Mme Marie-Pierre Rixain,Mme Marie-Pierre Rixain appelle l'attention de...
